In [19]:
#import dependencies
import pandas as pd
import json

In [20]:
# Read CSV files with low memory usage to handle large datasets
movies_metadata_csv = pd.read_csv("src/movies_metadata.csv", low_memory=False)
credits_part1_csv = pd.read_csv("src/credits_part1.csv", low_memory=False)
credits_part2_csv = pd.read_csv("src/credits_part2.csv", low_memory=False)
omdb_data_csv = pd.read_csv("src/omdb_data.csv", low_memory=False)

# Concatenate credits CSVs, remove duplicate entries based on 'id'
credits_csv = pd.concat([credits_part1_csv, credits_part2_csv]).drop_duplicates(subset='id')

# Rename 'imdbID' column to 'imdb_id' to match other DataFrames for merging
omdb_data_csv.rename(columns={"imdbID": "imdb_id"}, inplace=True)

# Remove duplicate rows in OMDB data based on 'imdb_id', keeping the first occurrence
omdb_data_csv.drop_duplicates(subset='imdb_id', keep='first', inplace=True)

# Clean the movies metadata:
# - Convert 'id' to numeric, setting invalid values to NaN
# - Convert 'release_date' to datetime, setting invalid values to NaT
# - Drop rows where 'id' is NaN
# - Remove duplicate rows based on 'id'
movies_metadata_csv = (
    movies_metadata_csv
    .assign(
        id=pd.to_numeric(movies_metadata_csv["id"], errors="coerce"),
        release_date=pd.to_datetime(movies_metadata_csv['release_date'], errors='coerce')
    )
    .dropna(subset=["id"])
    .drop_duplicates(subset='id')
)

# Merge movies metadata with credits DataFrame on 'id', ensuring 'id' is an integer
# Set 'id' as the index for the resulting DataFrame
merged_df = (
    pd.merge(movies_metadata_csv, credits_csv, on='id', how='left')
    .astype({'id': 'int'})
    .set_index('id')
)

# Merge the resulting merged DataFrame with OMDB data on 'imdb_id'
merged_df = pd.merge(merged_df, omdb_data_csv, on='imdb_id', how='left')



# Apply filters to the merged DataFrame:
# - Keep only English-language films
# - Only include films released on or after 1980
# - Ensure 'BoxOffice' and 'imdbRating' columns are not missing
# - Include only rows where the 'Type' is 'movie'
# - Filter films by specific ratings
merged_df = merged_df[
    (merged_df["original_language"] == "en") & 
    (merged_df["release_date"] >= "1980-01-01") & 
    (merged_df['BoxOffice'].notna()) & 
    (merged_df['imdbRating'].notna()) &  # Ensure imdbRating is available
    (merged_df["Type"] == 'movie') & 
    (merged_df["Rated"].isin([  # Keep only specific Ratings
        "R", "Not Rated", "PG-13", "PG", 
        "Unrated", "TV-14", "TV-MA", "G", 
        "TV-PG", "TV-G"
    ]))
]

# Drop columns that are not needed for analysis
merged_df.drop(columns=[
    'adult', 'belongs_to_collection', 'genres', 'homepage', 
    'original_title', 'popularity', 'production_companies', 
    'production_countries', 'Released', 'spoken_languages', 
    'status', 'tagline', 'video', 'Title', 'Runtime', 
    'Plot', 'poster_path', 'Ratings', 'Metascore', 
    'DVD', 'Production', 'Website', 'Response', 
    'Season', 'Episode', 'seriesID', 'totalSeasons', 
    'vote_average', 'vote_count', 'budget', 'revenue'
], inplace=True)

# Set 'imdb_id' as the index for the final DataFrame
merged_df.set_index('imdb_id', inplace=True)

# Display the final merged DataFrame .to_csv("merged_df.csv")
merged_df
